<a href="https://colab.research.google.com/github/Mateus-Pasqueta/Cuida-/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# Cuida+ - Sistema de Suporte Simplificado para Cuidadores e Idosos

# Instalação de dependências
!pip install -q google-adk gspread oauth2client

# Imports
import os
from datetime import date
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Configuração inicial
print("""
🧓👵 BEM-VINDO AO Cuida+ 👵🧓
O seu assistente de cuidados para idosos
----------------------------------------
""")

# --- Configuração da Planilha (mantida como antes) ---
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("cuida-mais-credenciais.json", scope)
client = gspread.authorize(creds)
sheet = client.open("cuida_mais_checkins").sheet1

# --- Funções dos Agentes Simplificadas ---
def call_agent_simple(agent, message):
    """Versão simplificada da função de chamada do agente"""
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message)])

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            return "\n".join(part.text for part in event.content.parts if part.text)
    return "Não foi possível obter uma resposta."

# Agente de Check-in Simplificado
def fazer_checkin():
    print("\n📝 Vamos fazer o check-in diário (responda com sim/não ou frases curtas):")

    perguntas = [
        "Dormiu bem hoje?",
        "Comeu normalmente?",
        "Está se sentindo bem de humor?",
        "Teve algum sintoma diferente?",
        "Conversou/compartilhou com alguém hoje?"
    ]

    respostas = []
    for pergunta in perguntas:
        resposta = input(f"{pergunta} ")
        respostas.append(resposta)

    # Salva na planilha
    hoje = date.today().strftime("%d/%m/%Y")
    sheet.append_row([hoje] + respostas)

    print("\n✅ Check-in registrado com sucesso!")

# Agente de Respostas Simplificado
def responder_duvida(pergunta):
    agente = Agent(
        name="agente_respostas_simples",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente que responde dúvidas sobre cuidados com idosos.
        Use linguagem SIMPLES, direta e acessível.
        Máximo de 2-3 frases por resposta.
        Seja empático e acolhedor.
        """,
        description="Agente de respostas simples para cuidadores e idosos"
    )

    resposta = call_agent_simple(agente, pergunta)
    return resposta

# Agente de Emergência Simplificado
def verificar_emergencia():
    ultima_linha = sheet.get_all_values()[-1]
    sintomas = ultima_linha[4]  # Assumindo que sintomas estão na coluna 4

    agente = Agent(
        name="agente_emergencia_simples",
        model="gemini-2.0-flash",
        instruction="""
        Analise os sintomas reportados e determine se é urgente.
        Responda em UMA FRASE apenas com:
        - "Procure ajuda médica" (se urgente)
        - "Tudo parece normal" (se não urgente)
        - "Fique atento" (se precisar monitorar)
        """,
        description="Detector de emergências simplificado"
    )

    resposta = call_agent_simple(agente, f"Sintomas: {sintomas}")
    return resposta

# --- Fluxo Principal Simplificado ---
# Cuida+ - Sistema de Suporte com Alegria! 🌟

def main():
    print("""
✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨
   BEM-VINDO AO Cuida+ 2.0!
     O sistema que ama cuidar!
✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨
""")

    while True:
        print("\n🎯 MENU PRINCIPAL - O que vamos fazer hoje?")
        print("1️⃣  | Fazer check-in diário 📅")
        print("2️⃣  | Tirar uma dúvida ❓")
        print("3️⃣  | Verificar como está o idoso 🩺")
        print("4️⃣  | Sair 🏁")

        opcao = input("\n🎨 Escolha uma opção (1-4): ")

        if opcao == "1":
            print("\n🌈 Vamos registrar como foi o dia!")
            fazer_checkin()
            print("\n🎉 Check-in concluído! Você é demais! 💪")

        elif opcao == "2":
            duvida = input("\n🤔 Qual é a sua dúvida sobre cuidados? Fala pra gente: ")
            resposta = responder_duvida(duvida)
            print(f"\n✨ RESPOSTA: {resposta}")
            print("\n🧠 Sabia que perguntar é sinal de cuidado excelente? Você arrasa! 👏")

        elif opcao == "3":
            status = verificar_emergencia()
            print(f"\n🔍 STATUS: {status}")
            if "ajuda" in status.lower():
                print("\n🚨 Vamos lá! Você consegue lidar com isso! Ligue para o médico e depois nos conte como foi! 💪")
            else:
                print("\n🌻 Tudo sob controle! Mais um dia de vitórias! 🏆")

        elif opcao == "4":
            print("""

💖💖💖💖💖💖💖💖💖💖💖💖💖💖💖💖
   MUITO OBRIGADO POR USAR O Cuida+!
   Você faz a diferença na vida de quem cuida!
   Volte sempre que precisar!
   Nosso lema: Cuidar com AMOR & ALEGRIA!
💖💖💖💖💖💖💖💖💖💖💖💖💖💖💖💖
            """)
            break

        else:
            print("\n⚠️ Oops! Opção inválida. Mas sem stress! Tenta de novo com 1, 2, 3 ou 4! 😊")

        # Mensagem motivacional aleatória após cada ação
        import random
        mensagens = [
            "\n💫 Lembre-se: cada cuidado seu vale ouro!",
            "\n🌱 Você está fazendo um trabalho incrível!",
            "\n🤗 O mundo precisa mais de cuidadores como você!",
            "\n🌟 Hoje você já foi a estrela de alguém!",
            "\n💕 Cuidar é o maior ato de amor!"
        ]
        print(random.choice(mensagens))

if __name__ == "__main__":
    main()


🧓👵 BEM-VINDO AO Cuida+ 👵🧓
O seu assistente de cuidados para idosos
----------------------------------------


✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨
   BEM-VINDO AO Cuida+ 2.0!
     O sistema que ama cuidar!
✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨


🎯 MENU PRINCIPAL - O que vamos fazer hoje?
1️⃣  | Fazer check-in diário 📅
2️⃣  | Tirar uma dúvida ❓
3️⃣  | Verificar como está o idoso 🩺
4️⃣  | Sair 🏁

🎨 Escolha uma opção (1-4): 1

🌈 Vamos registrar como foi o dia!

📝 Vamos fazer o check-in diário (responda com sim/não ou frases curtas):
Dormiu bem hoje? Sim
Comeu normalmente? Não
Está se sentindo bem de humor? Sim
Teve algum sintoma diferente? Não
Conversou/compartilhou com alguém hoje? Sim

✅ Check-in registrado com sucesso!

🎉 Check-in concluído! Você é demais! 💪

🌟 Hoje você já foi a estrela de alguém!

🎯 MENU PRINCIPAL - O que vamos fazer hoje?
1️⃣  | Fazer check-in diário 📅
2️⃣  | Tirar uma dúvida ❓
3️⃣  | Verificar como está o idoso 🩺
4️⃣  | Sair 🏁

🎨 Escolha uma opção (1-4): 3

🔍 STATUS: Tudo parece normal

🌻 Tudo sob controle